In [24]:
import pandas as pd
import requests

In [23]:
query_user = "What are the cheapest flights from London to Paris for 1 Adult leaving on the 25th September 2023, returning on the 30th September 2023 "

query = f'''Based on the user query about flights:{query_user}, respond with the following structure delimited by quotation marks as an example:
    
"
The response should be returned with the structure given for this for the example of a customer flying from London to Tokyo and back requesting the journey: 

I now have the 20 cheapest Journeys from London to Tokyo departing on the 30th of August 2023 and returning on the 15th of September 2023. 

|   Journey ID | Travel Direction   | Departure           | Arrival             | Journey Start   | Journey End   | Intermediate Departure   | Intermediate Arrival   | Airline    |   Flight Duration (hrs) | Total       |
|-------------:|:-------------------|:--------------------|:--------------------|:----------------|:--------------|:-------------------------|:-----------------------|:-----------|------------------------:|:------------|
|          167 | Outbound           | 2023-08-30T09:40:00 | 2023-08-31T10:40:00 | LHR             | HND           | N/A                      | CDG                    | AIR France |                     10.5 | 1422.79 EUR |
|          167 | Outbound           | 2023-08-31T10:40:00 | 2023-08-31T22:40:00 | LHR             | HND           | CDG                      | N/A                    | AIR France |                      3.5 |            |
|          167 | Inbound            | 2023-09-15T09:40:00 | 2023-09-16T05:35:00 | HND             | LHR           | N/A                      | CDG                    | AIR France |                     10.5 |            |
|          167 | Inbound            | 2023-09-16T06:35:00 | 2023-09-16T07:35:00 | HND             | LHR           | CDG                      | N/A                    | AIR France |                      3.5 |            |
|          168 | Outbound           | 2023-08-30T09:40:00 | 2023-08-31T22:40:00 | LHR             | HND           | N/A                      | N/A                    | Air Tokyo  |                     10.5 | 1550 EUR    |
|          168 | Inbound            | 2023-09-15T09:40:00 | 2023-09-16T07:35:00 | HND             | LHR           | n/a                      | N/A                    | Air Tokyo  |                      3.5 |            |

"
Return all legs for each Journey ID. when the requests mentions flight or flights, it really means Journey.

'''


In [25]:
# read keys
import yaml
import os

def read_config():
    # Get the directory of the current script
    # script_dir = os.path.dirname(os.path.realpath(__file__))
    script_dir = "../src/"

    # Construct the full path to the configuration file
    file_path = os.path.join(script_dir, "apikeys.yml")

    with open(file_path, 'r') as stream:
        try:
            configs = yaml.safe_load(stream)
            api_key = configs['amadeues_flights']['api_key']
            api_secret = configs['amadeues_flights']['api_secret']
            openai_key = configs['openai']['openai_key']
            hugging_api_key = configs['huggingfacehub']['hugging_api_key']
            return api_key, api_secret, openai_key, hugging_api_key
        except yaml.YAMLError as exc:
            print(exc)
            
    return api_key, api_secret, openai_key

class SingletonToken:
    __token = None

    @classmethod
    def set_token(cls, token):
        cls.__token = token

    @classmethod
    def get_token(cls):
        return cls.__token

api_key, api_secret, openai_key, hugging_api_key = read_config()

In [26]:
from dateutil.relativedelta import relativedelta
import re

def parse_duration(duration_str):
    # Extract hours and minutes from the duration string
    hours = re.search('(\d+)H', duration_str)
    minutes = re.search('(\d+)M', duration_str)
    
    # Convert hours and minutes to integers and calculate the total duration in hours
    hours = int(hours.group(1)) if hours else 0
    minutes = int(minutes.group(1)) if minutes else 0
    duration_in_hours = hours + minutes / 60

    return duration_in_hours

In [15]:
import numpy as np
def journey_data(response_flights_data, response_airline_lookup_data, originLocationCode, destinationLocationCode):
    # Load the data into a DataFrame
    df = pd.DataFrame(response_flights_data)
    df_airline_codes = pd.json_normalize(response_airline_lookup_data)
    
    # Extract itineraries, validatingAirlineCodes, price (total and currency) and id into separate dataframes
    df_itineraries = df[['id', 'itineraries']].explode('itineraries').reset_index(drop=True)
    
    # In the itineraries column, each cell is a dictionary. So, we need to convert those dictionaries into separate columns.
    df_itineraries = df_itineraries.join(pd.json_normalize(df_itineraries['itineraries'])).drop(columns='itineraries')
    
    # At this point, 'segments' column is a list of dictionaries where each dictionary represents a leg of the journey.
    # We want each leg to be a separate row in the dataframe. So, explode the 'segments' column.
    df_itineraries = df_itineraries.explode('segments').reset_index(drop=True)
    
    # Add a 'leg_id' column to identify each leg of the journey
    df_itineraries['leg_id'] = df_itineraries.groupby('id').cumcount() + 1
    
    # Now, convert the dictionaries in the 'segments' column into separate columns
    df_segments = pd.json_normalize(df_itineraries['segments'])
    
    # To avoid overlapping columns, add a prefix to the column names of the new dataframe
    df_segments.columns = ['flight_' + str(col) for col in df_segments.columns]
    
    # Now join the original dataframe with the new one
    df_itineraries = df_itineraries.join(df_segments).drop(columns='segments')
    
    df_validatingAirlineCodes = df[['id', 'validatingAirlineCodes']]
    
    # For the price column, we only need total and currency. So, extract only those into a new dataframe
    df_price = df['price'].apply(pd.Series)[['total', 'currency']]
    df_price['id'] = df['id']
    
    # Now join these dataframes on the 'id' column
    df_flights = pd.merge(df_itineraries, df_validatingAirlineCodes, on='id')
    df_flights = pd.merge(df_flights, df_price, on='id')
    
    # Create a new column for the total number of legs per journey
    df_flights['total_legs'] = df_flights.groupby('id')['leg_id'].transform('max')
    
    df_flights = df_flights.merge(right=df_airline_codes, how='left', left_on="flight_operating.carrierCode", right_on="iataCode")
    df_flights.rename(columns={"id":"journey_id", "commonName":"airline" }, inplace=True)

    df_flights.drop(columns=["flight_id", "validatingAirlineCodes", "businessName", "flight_operating.carrierCode", "flight_aircraft.code", "flight_stops"], inplace=True)

    df_flights.columns = df_flights.columns.str.replace('.', '_')
    df_flights['total'] = pd.to_numeric(df_flights['total'], errors='coerce')

    # convert duration into numeric form
    df_flights['flight_duration'] = np.round(df_flights['flight_duration'].apply(parse_duration),1)

    df_flights['flight_departure_at'] = pd.to_datetime(df_flights['flight_departure_at'])
    df_flights['flight_arrival_at'] = pd.to_datetime(df_flights['flight_arrival_at'])

    # calculate total duration 
    total_duration = df_flights.groupby('journey_id')['flight_duration'].sum().reset_index()
    total_duration.rename(columns={'flight_duration': 'total_duration'}, inplace=True)
    df_flights = pd.merge(df_flights, total_duration, on='journey_id')

    outbound_origin = originLocationCode
    outbound_destination = destinationLocationCode
    inbound_origin = destinationLocationCode
    inbound_destination = originLocationCode

    # Create conditions
    cond1 = (df_flights['flight_departure_iataCode'] == outbound_origin) | (df_flights['flight_arrival_iataCode'] == destinationLocationCode)
    cond2 = (df_flights['flight_departure_iataCode'] == inbound_origin) | (df_flights['flight_arrival_iataCode'] == inbound_destination)
    
    # Update 'Journey Start' and 'Journey End' based on conditions
    df_flights.loc[cond1, 'Journey Start'] = originLocationCode
    df_flights.loc[cond1, 'Journey End'] = destinationLocationCode
    df_flights.loc[cond2, 'Journey Start'] = destinationLocationCode
    df_flights.loc[cond2, 'Journey End'] = originLocationCode
    
    # Update 'travel_direction' based on 'Journey Start'
    df_flights.loc[df_flights['Journey Start'] == originLocationCode, 'travel_direction'] = 'Inbound'
    df_flights.loc[df_flights['Journey Start'] == destinationLocationCode, 'travel_direction'] = 'Outbound'

    df_flights.loc[df_flights['flight_arrival_iataCode'] == df_flights['Journey End'], 'flight_arrival_iataCode'] = 'N/A'
    df_flights.loc[df_flights['flight_departure_iataCode'] == df_flights['Journey Start'], 'flight_departure_iataCode'] = 'N/A'

    df_flights.rename(columns={'flight_departure_iataCode': 'intermediate_journey_departure', 
                           'flight_arrival_iataCode': 'intermediate_journey_arrival'}, inplace=True)

    journey_pricing = df_flights[['journey_id', 'Journey Start', 'Journey End', 'travel_direction', 'total_duration', 'total']].drop_duplicates()
    flights = df_flights.drop(columns=['Journey Start', 'Journey End', 'travel_direction', 'total_duration', 'total', 'duration'])

    return df_flights, journey_pricing, flights

In [16]:
from sqlalchemy import create_engine
import pandas as pd
from langchain import SQLDatabase

def load_data(journey_pricing, flights):
    engine = create_engine('sqlite:///:memory:')

    # Write the data to the SQLite database
    flights.to_sql('flights', engine, if_exists='replace', index=False)
    journey_pricing.to_sql('journey_pricing', engine, if_exists='replace', index=False)
    # Check if the data was loaded correctly
    df_loaded = pd.read_sql('SELECT * FROM flights', engine)
    db = SQLDatabase(engine)
    return db

In [27]:
import json
import openai
from langchain.tools import tool

def get_args(query_user: str) -> str:
    # OpenAI function calling

    """Get's arguments based on client query, 
    returns num_adults, departureDate, returnDate destinationLocationCode, originLocationCode.
    This is required before pulling the data from the API.
    """
    
    function_call = [
    {
      "name": "search_for_flights",
      "description": "Requests flight data from Amadeus API and writes to SQLite database",
      "parameters": {
        "type": "object",
        "properties": {
            "num_adults":{
                "type":"integer",
                "description": '''Based on the query, respond with the number of adults'''
            },
            "departureDate": {
                "type":"string",
                "description": '''Based on the query, respond with the Departure Date. Dates are specified in the ISO 8601 YYYY-MM-DD format. '''
            },
            "returnDate": {
                "type":"string",
                "description": '''Based on the query, respond with the Return Date. Dates are specified in the ISO 8601 YYYY-MM-DD format. '''
            },
            "destinationLocationCode":{
                "type":"string",
                "description": '''Based on the query, respond with an airport IATA code from the city which the traveler is going. E.g CDG for Charles de Gaulle Airport'''
            },
          "originLocationCode": {
            "type": "string",
            "description": '''Based on the query, respond with an airport IATA code from the city which the traveler will depart from. E.g CDG for Charles de Gaulle Airport'''
          },

        },
        "required": ["destinationLocationCode", "originLocationCode", "departureDate", "returnDate", "num_adults"]
      }
    }
    ]
    
    openai.api_key = openai_key

    message = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[{"role": "user", "content": query_user}],
        functions = function_call,
        function_call = 'auto',
        temperature=0
    )
    response_message = message["choices"][0]["message"]["function_call"]["arguments"]

    parsed_data = json.loads(response_message)

    # Accessing variables
    num_adults = parsed_data['num_adults']
    departureDate = parsed_data['departureDate']
    returnDate = parsed_data['returnDate']
    destinationLocationCode = parsed_data['destinationLocationCode']
    originLocationCode = parsed_data['originLocationCode']
    
    print("Number of Adults: ", num_adults)
    print("Departure Date: ", departureDate)
    print("Return Date: ", returnDate)
    print("Destination Location Code: ", destinationLocationCode)
    print("Origin Location Code: ", originLocationCode)

    return num_adults, departureDate, returnDate, destinationLocationCode, originLocationCode

num_adults, departureDate, returnDate, destinationLocationCode, originLocationCode = get_args(query_user)

Number of Adults:  1
Departure Date:  2023-09-25
Return Date:  2023-09-30
Destination Location Code:  CDG
Origin Location Code:  LHR


In [18]:
from amadeus import Client, ResponseError
from datetime import datetime
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.tools.python.tool import PythonREPLTool
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain.tools import tool
from langchain import LLMMathChain
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

def search_for_flights(originLocationCode, destinationLocationCode, departureDate, returnDate, num_adults) -> SQLDatabase:
    """Requests flight data from Amadeus API and writes to sqllite database and run SQLDatabaseQuery
    originLocationCode: Based on the query, respond with the iataCode for the origin airport,
    destinationLocationCode: Based on the query, respond with the iataCode for the destination airport,
    departureDate: Based on the query, respond with the departure date,
    num_adults: Based on the query, respond with the number of adults
    """

    # Assuming you've defined api_key and api_secret somewhere else
    amadeus = Client(client_id=api_key, client_secret=api_secret)

    # Defining the parameters for the flight
    params = {
        'originLocationCode': originLocationCode,
        'destinationLocationCode': destinationLocationCode,
        'departureDate': departureDate,
        'returnDate': returnDate,
        'adults': num_adults
    }
    
    try:
        response_flights = amadeus.shopping.flight_offers_search.get(**params)
        
    except ResponseError as error:
        print(f"ResponseError occurred flights: {error}")
        print(f"Error code flights: {error.code}")
        print(f"Error message flights: {error.description}")
        return []  # return an empty list in case of an error

    try:
        response_airline_lookup = amadeus.reference_data.airlines.get()

    except ResponseError as error:
        print(f"ResponseError occurred airline lookup: {error}")
        print(f"Error code airline lookup: {error.code}")
        print(f"Error message airline lookup: {error.description}")

    df_flights, journey_pricing, flights = journey_data(response_flights.data, response_airline_lookup.data, destinationLocationCode, originLocationCode)
    print(df_flights.dtypes)
    db = load_data(journey_pricing, flights)

    return db, df_flights, journey_pricing, flights

db, df_flights, journey_pricing, flights = search_for_flights(originLocationCode, destinationLocationCode, departureDate, returnDate, num_adults)
journey_pricing

journey_id                                object
duration                                  object
leg_id                                     int64
flight_carrierCode                        object
flight_number                             object
flight_duration                          float64
flight_numberOfStops                       int64
flight_blacklistedInEU                      bool
intermediate_journey_departure            object
flight_departure_terminal                 object
flight_departure_at               datetime64[ns]
intermediate_journey_arrival              object
flight_arrival_terminal                   object
flight_arrival_at                 datetime64[ns]
total                                    float64
currency                                  object
total_legs                                 int64
type                                      object
iataCode                                  object
icaoCode                                  object
airline             

,journey_id,Journey Start,Journey End,travel_direction,total_duration,total
0,1,LHR,HND,Outbound,30.5,1049.34
2,1,HND,LHR,Inbound,30.5,1049.34
4,2,LHR,HND,Outbound,30.3,1049.34
6,2,HND,LHR,Inbound,30.3,1049.34
8,3,LHR,HND,Outbound,34.2,1049.34
...,...,...,...,...,...,...
975,248,HND,LHR,Inbound,14.1,8658.42
976,249,LHR,HND,Outbound,14.1,8658.42
978,249,HND,LHR,Inbound,14.1,8658.42
979,250,LHR,HND,Outbound,13.9,8658.42


In [9]:
flights

,journey_id,leg_id,flight_carrierCode,flight_number,flight_duration,flight_numberOfStops,flight_blacklistedInEU,intermediate_journey_departure,flight_departure_terminal,flight_departure_at,intermediate_journey_arrival,flight_arrival_terminal,flight_arrival_at,currency,total_legs,type,iataCode,icaoCode,airline
0,1,1,CX,250,12.1,0,False,N/A,3,2023-08-30 18:20:00,HKG,1,2023-08-31 13:25:00,EUR,4,airline,CX,CPA,CATHAYPACIFIC
1,1,2,CX,235,0.5,0,False,HKG,1,2023-08-31 16:20:00,N/A,3,2023-08-31 17:50:00,EUR,4,airline,CX,CPA,CATHAYPACIFIC
2,1,3,CX,543,4.8,0,False,N/A,3,2023-09-15 10:05:00,HKG,1,2023-09-15 13:55:00,EUR,4,airline,CX,CPA,CATHAYPACIFIC
3,1,4,CX,257,13.1,0,False,HKG,1,2023-09-16 09:10:00,N/A,3,2023-09-16 15:15:00,EUR,4,airline,CX,CPA,CATHAYPACIFIC
4,2,1,CX,238,11.9,0,False,N/A,3,2023-08-30 17:00:00,HKG,1,2023-08-31 11:55:00,EUR,4,airline,CX,CPA,CATHAYPACIFIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,249,2,CX,235,0.5,0,False,HKG,1,2023-08-31 16:20:00,N/A,3,2023-08-31 17:50:00,EUR,3,airline,CX,CPA,CATHAYPACIFIC
978,249,3,NH,211,1.5,0,False,N/A,2,2023-09-15 11:30:00,N/A,2,2023-09-15 05:00:00,EUR,3,airline,NH,ANA,ALL NIPPON
979,250,1,CX,238,11.9,0,False,N/A,3,2023-08-30 17:00:00,HKG,1,2023-08-31 11:55:00,EUR,3,airline,CX,CPA,CATHAYPACIFIC
980,250,2,CX,235,0.5,0,False,HKG,1,2023-08-31 16:20:00,N/A,3,2023-08-31 17:50:00,EUR,3,airline,CX,CPA,CATHAYPACIFIC


In [10]:
# from langchain.prompts.prompt import PromptTemplate

# _DEFAULT_TEMPLATE = """Based on the user query about flights:{query_user}, respond with the following structure delimited by quotation marks as an example:
    
# "
# The response should be returned like this for the example of a customer flying from London to Tokyo and back: 

# I now have the 5 cheapest Journeys from London to Tokyo departing on the 30th of August 2023 and returning on the 15th of September 2023. 

# | Journey ID   |   Leg ID | Outbound Departure   | Outbound Arrival    | Return Departure    | Return Arrival      | Dep Airport   | Arrival Airport   | Airline    | Total       |
# |:-------------|---------:|:---------------------|:--------------------|:--------------------|:--------------------|:--------------|:------------------|:-----------|:------------|
# | 167.0        |        1 | 2023-08-30T09:40:00  | 2023-08-31T10:40:00 | N/A                 | N/A                 | LHR           | CDG               | AIR France | 1422.79 EUR |
# |              |        2 | 2023-08-31T10:40:00  | 2023-08-31T22:40:00 | N/A                 | N/A                 | CDG           | HND               | AIR France | 1422.79 EUR |
# |              |        1 | N/A                  | N/A                 | 2023-09-15T09:40:00 | 2023-09-16T05:35:00 | HND           | CDG               | AIR France | 1422.79 EUR |
# |              |        2 | N/A                  | N/A                 | 2023-09-16T06:35:00 | 2023-09-16T07:35:00 | CDG           | LHR               | AIR France | 1422.79 EUR |
# | 168.0        |        1 | 2023-08-30T09:40:00  | 2023-08-31T22:40:00 | N/A                 | N/A                 | LHR           | HND               | Air Tokyo  | 1550 EUR    |
# |              |        1 | N/A                  | N/A                 | 2023-09-15T09:40:00 | 2023-09-16T07:35:00 | HND           | LHR               | Air Tokyo  | 1550 EUR    |

# "
# Journeys can have multiple legs as denoted by their leg_id. If you wanted to get the cheapest Journey, do not filter based on the flight destination. All journeys have already been prefiltered.

# For example:
# Tokyo to London could be 2 two legs HND to CDG then CDG to LHR. It could also be HND to CDG. Filtering for departure = CDG and destination = LHR
# will lead you to miss Journeys with two legs. Return ALL leg_id for each Journey ID. Therefore, getting the cheapest journey would be just sorting the table by ascending price.

# when the requests says flight, it really means journey.

# '''"""
# PROMPT = PromptTemplate(
#     input_variables=["query_user"], template=_DEFAULT_TEMPLATE
# )

In [11]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(temperature=0, model="gpt-4-0613", openai_api_key=openai_key)

def find_flights(query, llm):
    '''creates agent that can be run on db to answer query flights'''
    llm=llm
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    agent_executor = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    )
 
    return agent_executor.run(query)


response = find_flights(query, llm)
response



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: flights, journey_pricing
Thought:The tables 'flights' and 'journey_pricing' seem relevant to the question. I should check their schemas to understand the structure and the data they contain.
Action: sql_db_schema
Action Input: "flights, journey_pricing"
Observation: 
CREATE TABLE flights (
	journey_id TEXT, 
	leg_id BIGINT, 
	"flight_carrierCode" TEXT, 
	flight_number TEXT, 
	flight_duration FLOAT, 
	"flight_numberOfStops" BIGINT, 
	"flight_blacklistedInEU" BOOLEAN, 
	intermediate_journey_departure TEXT, 
	flight_departure_terminal TEXT, 
	flight_departure_at DATETIME, 
	intermediate_journey_arrival TEXT, 
	flight_arrival_terminal TEXT, 
	flight_arrival_at DATETIME, 
	currency TEXT, 
	total_legs BIGINT, 
	type TEXT, 
	"iataCode" TEXT, 
	"icaoCode" TEXT, 
	airline TEXT
)

/*
3 rows from flights table:
journey_id	leg_id	flight_carrierCode	flight_number	flight_duration	flight_numberOfStops	fli

'"\nI now have the 10 cheapest Journeys from London to Tokyo departing on the 30th of August 2023 and returning on the 15th of September 2023. \n\n|   Journey ID | Travel Direction   | Departure           | Arrival             | Journey Start   | Journey End   | Intermediate Departure   | Intermediate Arrival   | Airline    |   Flight Duration (hrs) | Total       |\n|-------------:|:-------------------|:--------------------|:--------------------|:----------------|:--------------|:-------------------------|:-----------------------|:-----------|------------------------:|:------------|\n|          1 | Outbound           | 2023-08-31T16:20:00 | 2023-08-31T17:50:00 | LHR             | HND           | HKG                      | N/A                    | CATHAYPACIFIC |                     0.5 | 1049.34 EUR |\n|          1 | Outbound           | 2023-08-30T18:20:00 | 2023-08-31T13:25:00 | LHR             | HND           | N/A                      | HKG                    | CATHAYPACIFIC |     

In [12]:
df_flights[["journey_id", "flight_departure_iataCode", "flight_arrival_iataCode", "Journey Start", "Journey End", "travel_direction"]].loc[df_flights['journey_id'] == '180']

KeyError: "['flight_departure_iataCode', 'flight_arrival_iataCode'] not in index"

In [ ]:
# Create conditions
cond1 = (df_flights['flight_departure_iataCode'] == outbound_origin) | (df_flights['flight_arrival_iataCode'] == destinationLocationCode)
cond2 = (df_flights['flight_departure_iataCode'] == inbound_origin) | (df_flights['flight_arrival_iataCode'] == inbound_destination)

# Update 'Journey Start' and 'Journey End' based on conditions
df_flights.loc[cond1, 'Journey Start'] = originLocationCode
df_flights.loc[cond1, 'Journey End'] = destinationLocationCode
df_flights.loc[cond2, 'Journey Start'] = destinationLocationCode
df_flights.loc[cond2, 'Journey End'] = originLocationCode

# Update 'travel_direction' based on 'Journey Start'
df_flights.loc[df_flights['Journey Start'] == originLocationCode, 'travel_direction'] = 'Inbound'
df_flights.loc[df_flights['Journey Start'] == destinationLocationCode, 'travel_direction'] = 'Outbound'


In [ ]:
df_flights.sort_values(by='total', ascending=True)

In [19]:
df_flights

,journey_id,duration,leg_id,flight_carrierCode,flight_number,flight_duration,flight_numberOfStops,flight_blacklistedInEU,intermediate_journey_departure,flight_departure_terminal,...,currency,total_legs,type,iataCode,icaoCode,airline,total_duration,Journey Start,Journey End,travel_direction
0,1,PT15H30M,1,CX,250,12.1,0,False,N/A,3,...,EUR,4,airline,CX,CPA,CATHAYPACIFIC,30.5,LHR,HND,Outbound
1,1,PT15H30M,2,CX,235,0.5,0,False,HKG,1,...,EUR,4,airline,CX,CPA,CATHAYPACIFIC,30.5,LHR,HND,Outbound
2,1,PT37H10M,3,CX,543,4.8,0,False,N/A,3,...,EUR,4,airline,CX,CPA,CATHAYPACIFIC,30.5,HND,LHR,Inbound
3,1,PT37H10M,4,CX,257,13.1,0,False,HKG,1,...,EUR,4,airline,CX,CPA,CATHAYPACIFIC,30.5,HND,LHR,Inbound
4,2,PT16H50M,1,CX,238,11.9,0,False,N/A,3,...,EUR,4,airline,CX,CPA,CATHAYPACIFIC,30.3,LHR,HND,Outbound
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,249,PT15H30M,2,CX,235,0.5,0,False,HKG,1,...,EUR,3,airline,CX,CPA,CATHAYPACIFIC,14.1,LHR,HND,Outbound
978,249,PT1H30M,3,NH,211,1.5,0,False,N/A,2,...,EUR,3,airline,NH,ANA,ALL NIPPON,14.1,HND,LHR,Inbound
979,250,PT16H50M,1,CX,238,11.9,0,False,N/A,3,...,EUR,3,airline,CX,CPA,CATHAYPACIFIC,13.9,LHR,HND,Outbound
980,250,PT16H50M,2,CX,235,0.5,0,False,HKG,1,...,EUR,3,airline,CX,CPA,CATHAYPACIFIC,13.9,LHR,HND,Outbound


In [21]:
import os
import plantuml

# Set PlantUML path
os.environ["PLANTUML"] = "/path/to/plantuml.jar"

# Create PlantUML object
puml = plantuml.PlantUML()

# Generate diagram
puml.processes_file("classes_FrontEnd.puml")


TypeError: __init__() missing 1 required positional argument: 'url'

In [22]:
import subprocess

# Path to your PlantUML jar file
plantuml_path = "/path/to/plantuml.jar"
# Path to your .puml file
puml_file = "classes_FrontEnd.puml"
# Output format (png, svg, etc.)
output_format = "png"

# Command to generate the diagram
command = f"java -jar {plantuml_path} -t{output_format} {puml_file}"

# Run the command
subprocess.run(command, shell=True)


CompletedProcess(args='java -jar /path/to/plantuml.jar -tpng classes_FrontEnd.puml', returncode=1)